# Exploratory Data Analysis

This dataset is taken from a fantastic paper that looks to see how analytical choices made by different data science teams on the same dataset in an attempt to answer the same research question affect the final outcome.

[Many analysts, one dataset: Making transparent how variations in analytical choices affect results](https://osf.io/gvm2z/)

The data can be found [here](https://osf.io/47tnc/).



# The Task

Take the dataframe `dfd` above and do an Exploratory Data Analysis on the dataset. Keeping in mind the question is the following: **Are soccer referees more likely to give red cards to dark skin toned players than light skin toned players?**

Please work on your own -- and submit a pull-request when you have completed it. We are interested in how the Data Scientists approach EDA and will compile the results into a larger document.

No need to be elaborate w/ explanation; bullet-points are fine.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
# %load_ext autoreload
# the "1" means: always reload modules marked with "%aimport"
# %autoreload 1

from __future__ import absolute_import, division, print_function
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import GridSpec
import seaborn as sns
import mplsvds
import mpld3
import numpy as np
import pandas as pd
import os, sys
from tqdm import tqdm
import warnings

sns.set_context("poster", font_scale=1.3)

In [2]:
# # The code from this cell comes from: https://osf.io/w7tds/

# import pandas as pd #for dealing with csv import
# import os # for joining paths and filenames sensibly

# # print "loading datafiles"
# # nb new datase
# filename = os.path.join('../data/redcard/','CrowdstormingDataJuly1st.csv') 
# df = pd.read_csv(filename)

# #add new vars
# df['skintone']=(df['rater1']+df['rater2'])/2
# df['allreds']=df['yellowReds']+df['redCards']
# df['allredsStrict']=df['redCards']
# df['refCount']=0

# #add a column which tracks how many games each ref is involved in
# refs=pd.unique(df['refNum'].values.ravel()) #list all unique ref IDs

# #for each ref, count their dyads
# for r in refs:
#     df['refCount'][df['refNum']==r]=len(df[df['refNum']==r])    

# colnames=list(df.columns)

# j = 0
# out = [0 for _ in range(sum(df['games']))]

# for _, row in df.iterrows():
#         n = row['games']
#         c = row['allreds']
#         d = row['allredsStrict']
        
#         #row['games'] = 1        
        
#         for _ in range(n):
#                 row['allreds'] = 1 if (c-_) > 0 else 0
#                 row['allredsStrict'] = 1 if (d-_) > 0 else 0
#                 rowlist=list(row)  #convert from pandas Series to prevent overwriting previous values of out[j]
#                 out[j] = rowlist
#                 j += 1


# pd.DataFrame(out, columns=colnames).to_csv('../data/redcard/crowdstorm_disaggregated.csv', index=False)

In [13]:
dfd = pd.read_csv("../data/redcard/crowdstorm_disaggregated.csv.gz", compression='gzip')

## Data Structure

> The dataset is available as a list with 146,028 dyads of players and referees and includes details from players, details from referees and details regarding the interactions of player-referees. A summary of the variables of interest can be seen below. A detailed description of all variables included can be seen in the README file on the project website. -- https://docs.google.com/document/d/1uCF5wmbcL90qvrk_J27fWAvDcDNrO9o_APkicwRkOKc/edit


| Variable Name: | Variable Description: | 
| -- | -- | 
| playerShort | short player ID | 
| player | player name | 
| club | player club | 
| leagueCountry | country of player club (England, Germany, France, and Spain) | 
| height | player height (in cm) | 
| weight | player weight (in kg) | 
| position | player position | 
| games | number of games in the player-referee dyad | 
| goals | number of goals in the player-referee dyad | 
| yellowCards | number of yellow cards player received from the referee | 
| yellowReds | number of yellow-red cards player received from the referee | 
| redCards | number of red cards player received from the referee | 
| photoID | ID of player photo (if available) | 
| rater1 | skin rating of photo by rater 1 | 
| rater2 | skin rating of photo by rater 1 | 
| refNum | unique referee ID number (referee name removed for anonymizing purposes) | 
| refCountry | unique referee country ID number | 
| meanIAT | mean implicit bias score (using the race IAT) for referee country | 
| nIAT | sample size for race IAT in that particular country | 
| seIAT | standard error for mean estimate of race IAT   | 
| meanExp | mean explicit bias score (using a racial thermometer task) for referee country | 
| nExp | sample size for explicit bias in that particular country | 
| seExp |  standard error for mean estimate of explicit bias measure | 



In [14]:
dfd.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,skintone,allreds,allredsStrict,refCount
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.326391,712.0,0.000564,0.396000,750.0,0.002696,0.375,0,0,1
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.203375,40.0,0.010875,-0.204082,49.0,0.061504,0.750,0,0,1
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,...,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,NaN,0,0,3
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,...,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,NaN,0,0,3
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,...,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,NaN,0,0,3


In [15]:
dfd.shape

(426572, 32)